In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Databricks Model Serving

In [2]:
from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(endpoint="databricks-gpt-oss-120b")

In [3]:
chat_model.invoke("What is MLflow?")

/Users/nebula/Developer/GitHub/Lakehouse-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[{'type': 'reasoning', 's...grade ML pipelines.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


AIMessage(content=[{'type': 'reasoning', 'summary': [{'type': 'summary_text', 'text': 'The user asks "What is MLflow?" Provide an explanation. Explain MLflow as an open-source platform for managing ML lifecycle: tracking experiments, packaging code, sharing, deploying. Mention components: Tracking, Projects, Models, Model Registry. Provide details, use cases, architecture, features, comparison, maybe example code. Should be concise but thorough.'}]}, {'type': 'text', 'text': '**MLflow** is an open‑source platform (originally created by Databricks in 2018) that helps data scientists and engineers manage the entire **machine‑learning lifecycle**—from experimentation to production deployment—without locking you into any particular library, language, or cloud provider.\n\n---\n\n## Core Goals\n\n| Goal | Why it matters |\n|------|----------------|\n| **Reproducibility** | Capture code, data version, parameters, and environment so anyone can rerun a result. |\n| **Collaboration** | Centrali

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a chatbot that can answer questions about {topic}.",
        ),
        ("user", "{question}"),
    ]
)

chain = prompt | chat_model
output = chain.invoke(
    {
        "topic": "Databricks",
        "question": "What is Unity Catalog?",
    }
)

output.content

/Users/nebula/Developer/GitHub/Lakehouse-RAG/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[{'type': 'reasoning', 's...ance with Terraform)."}], input_type=list])
  return self.__pydantic_serializer__.to_python(


[{'type': 'reasoning',
  'summary': [{'type': 'summary_text',
    'text': 'We need to answer about Unity Catalog. Provide explanation: what it is, purpose, features, architecture, security, data governance, access control, integration with Databricks, tables, schemas, catalogs, fine-grained permissions, data lineage, sharing, external locations, support for Delta Lake, etc. Also possibly mention differences to older Hive metastore, migration, best practices, roles. Provide concise yet comprehensive answer.'}]},
 {'type': 'text',
  'text': "**Unity Catalog** is Databricks’ unified, cloud‑native data governance solution. It provides a single logical “catalog” that spans all of your data assets—files, tables, machine‑learning models, and notebooks—across **all** Databricks workspaces, regions, and clouds that belong to the same account.  \n\nBelow is a quick‑reference overview of what Unity Catalog is, why it exists, and the core concepts you’ll work with.\n\n---\n\n## 1. The Problem Unit